In [1]:
import numpy as np
import pandas as pd
import sys
import copy
import abc

from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV, RidgeCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from pandas.api.types import is_integer_dtype
from numpy.random import default_rng
from scipy.special import expit
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib widget

SeaVan1 :
X is drawn uniformly from {0,1,2}
Y|X = x ~ N(x, 1)
R|X = expit(4-4x) where expit(x) = 1.(1+e^-x)

In [2]:
np.random.seed(2021)
sv1 = pd.DataFrame({"X" : np.random.randint(0,3,size=50000)})
sv1["Y"] = sv1["X"].apply(lambda x : np.random.normal(x))
sv1["R"] = sv1["X"].apply(lambda x : 0 if 1/(1+np.exp(4*x-4)) < 0.5 else 1)
sv1.tail(5)

,X,Y,R
49995,2,3.670323,0
49996,0,1.520434,1
49997,0,0.431790,1
49998,1,1.246051,1
49999,0,1.029791,1


SeaVan2: X is drawn uniformly from {0,1,2}
Y|X = x ~ N(1[x>=1], 1)
R|X = expit(4-4x) where expit(x) = 1.(1+e^-x)

In [3]:
np.random.seed(2022)
sv2 = pd.DataFrame({"X" : np.random.randint(0,3,size=50000)})
sv2["Y"] = sv2["X"].apply(lambda x : np.random.normal(1 if x>=1 else 0))
sv2["R"] = sv2["X"].apply(lambda x : 0 if 1/(1+np.exp(4*x-4)) < 0.5 else 1)
sv2.tail(5)

,X,Y,R
49995,2,0.991530,0
49996,1,2.674972,1
49997,2,1.871297,0
49998,0,-0.256136,1
49999,0,0.549157,1


### Applying Baseline Models

Direct Methods : Linear Regression / Non-linear Regression / Regression Tree
MAR estimates : IPW , SN-IPW, IW, SN-IW, lin-impute, NL impute

SeaVan1

In [4]:
train, test = train_test_split(sv1, test_size=0.2, random_state=2021)

In [5]:
lr = LinearRegression()
obs_train_x = train.loc[train.R==1][["X"]]
obs_train_y = train.loc[train.R==1]["Y"]

lr.fit(obs_train_x, obs_train_y)

LinearRegression()

In [6]:
test_x = test[["X"]]
test_y = test["Y"]
pred_y = lr.predict(test_x)

def calc_rmse(pred, label):
    return np.sum(np.abs(pred-label))**0.5
rmse = calc_rmse(pred_y, test_y)

rmse

89.57631143879968

SeaVan2

### Load UCI datasets

In [22]:
yeast = pd.read_csv("yeast.csv")
yeast_data = yeast.drop("Sequence Name", axis=1)

bean = pd.read_csv("dry_bean.csv")
bean_data = bean.rename(columns={"Class": "label"})

letter_data = pd.read_csv('letter-recognition.data', names = ['label']+[f'x{i}' for i in range(16)])

In [8]:
def create_rewards(df, y_col, drop_context=False):
    
    classes = df[y_col].unique()
    contexts = df.drop(y_col, axis=1).columns
    df = pd.concat([df, pd.DataFrame(columns=classes)], axis=1).fillna(0)
    
    def fill_in_rewards(row, classes):
        label = row[y_col]
        row.loc[label] = 1
        return row
    
    df = df.apply(fill_in_rewards, axis=1, args=[classes])
    if drop_context:
        df = df.drop(contexts, axis=1)
    
    return df

### Setting up Target Policy

In [9]:
class Policy:
    def __init__(self, num_actions=2):
        self.num_actions = num_actions

    @abc.abstractmethod
    def get_action_distribution(self, X):
        raise NotImplementedError("Must override method")

    def get_action_propensities(self, X, actions):
        distrib = self.get_action_distribution(X)
        distrib["action"] = actions
        return distrib.apply(lambda x : x[x["action"]], axis=1)
        
    def select_actions(self, X, rng=default_rng(1)):
        df = self.get_action_distribution(X)
        action_list = df.columns
        
        df["actions"] = df.apply(lambda x : np.random.choice(action_list, p=x), axis=1)
        actions = df["actions"]
        propensities = self.get_action_propensities(X, actions)
        
        return actions, propensities
        
    def get_value_estimate(self, X, full_rewards):
        actions, propensities = self.select_actions(X)
        df = pd.DataFrame(full_rewards.reset_index(drop=True)).assign(act = actions)
        action_rewards = df.apply(lambda x : x[x["act"]], axis=1)
        
        return action_rewards.mean()

In [10]:
class SKLearnPolicy(Policy):
    """ 
    An SKLearnPolicy uses a scikit learn model to generate an action distribution.  If the SKLearnPolicy is built with is_deterministic=False, 
    then the predict distribution for a context x should be whatever predict_proba for the model returns.  If is_deterministic=True, then all the probability mass 
    should be concentrated on whatever predict of the model returns.
    """
    def __init__(self, model, num_actions=2, is_deterministic=False):
        self.is_deterministic = is_deterministic
        self.num_actions = num_actions
        self.model = model

    def get_action_distribution(self, X):
        prob = pd.DataFrame(self.model.predict_proba(X), columns=self.model.classes_)
        action = self.model.predict(X)
        
        def deterministic(row):
            pred = row["act"]
            row[pred] = 1
            return row
        
        if (self.is_deterministic):
            df = pd.DataFrame(np.zeros(prob.shape), columns=self.model.classes_).assign(act=action)
            df = df.apply(deterministic, axis=1).drop("act", axis=1)
        else:
            df = copy.deepcopy(prob)

        return df

    def select_actions(self, X, rng=default_rng(1)):
        if (self.is_deterministic):
            actions = pd.DataFrame(self.model.predict(X))
            propensities = pd.Series([1 for i in range(len(actions))])
        else:
            df = self.get_action_distribution(X)
            action_list = df.columns
            df["actions"] = df.apply(lambda x : np.random.choice(action_list, p=x), axis=1)
            actions = df["actions"]
            propensities = self.get_action_propensities(X, actions)
            
        return actions, propensities

class BanditLoggingPolicy(Policy):
    """
    This policy derives from another deterministic policy following the recipe described in the Vlassis et al paper, on the top of the second column in section 5.3.
    For any context x, if the deterministic policy selects action a, then this policy selects action a with probability eps (a supplied parameter), and spreads the
    rest of the probability mass uniformly over the other actions.
    """
    def __init__(self, num_actions=2, eps=0.3, actions=None, classes=None):
        self.num_actions = num_actions
        self.eps = eps
        self.actions = actions.reset_index(drop=True)
        self.classes = classes
        
    def get_action_distribution(self, X):
        
        def bandit_sampling(row, classes):
            pred = row["act"]
            s_a = np.random.uniform(0.1, 1, len(self.classes))
            row[row.index!="act"] +=  s_a/s_a.sum()*(1-self.eps) # take care of actions
            row[pred] += self.eps
            return row
        
        df = pd.DataFrame(np.zeros([X.shape[0], self.num_actions]), columns=self.classes).assign(act=self.actions)
        df = df.apply(bandit_sampling, axis=1, args=[self.classes]).drop("act", axis=1)
        return df

In [11]:
def generate_bandit_feedback(contexts, full_rewards, policy, rng=default_rng(1)):
    """   
    Args:
        contexts (np.array): contexts, rows correspond to entries of rewards
        full_rewards (np.array): 2-dim numpy array with the same number of rows as X and number of columns corresponding to the number actions
            each row gives the reward that would be received for each action for the context in the corresponding row of X. 

    Returns:
        new_contexts (np.array): new_n rows and same number of columns as in contexts
        actions (np.array): vector with new_n entries giving actions selected by the provided policy for the contexts in new_contexts
        observed_rewards (np.array): vector with new_n entries giving actions selected by the provided policy for the contexts in new_contexts 
    """   
    
    n, k = full_rewards.shape
    new_contexts = contexts
    actions, propensities = policy.select_actions(X=new_contexts, rng=rng)
    obs_rewards = full_rewards.reset_index(drop=True).assign(act=actions)
    obs_rewards["obs_r"] = obs_rewards.apply(lambda x : x[x["act"]], axis=1)
    observed_rewards = obs_rewards["obs_r"]
    return new_contexts, actions, observed_rewards, propensities

### Creating value estimators

In [27]:
def get_value_estimators(policy, contexts, actions, rewards, propensities, skip_slow_stuff=False):
    """   
    Args:
        policy (Policy): the policy we want to get a value estimate for
        contexts (np.array): contexts from bandit feedback
        actions (np.array): actions chosen for bandit feedback
        rewards (np.array): rewards received in bandit feedback
        propensities (np.array): the propensity for each action selected under the logging policy (which is not provided to this function)
        skip_slow_stuff (boolean): boolean flag which allows you to turn on/off some slow estimators (ignore this if you like)
    Returns:
        est (dict): keys are string describing the value estimator, values are the corresponding value estimates 
    """   

    est = {}
    est["mean"] = np.mean(rewards)
    new_propensities = policy.get_action_propensities(contexts, actions)
    imp_wgt = new_propensities / propensities
    
    est["iw"] = np.mean(rewards*imp_wgt)
    est["sn-iw"] = np.sum(rewards*imp_wgt) / np.sum(imp_wgt)

    merged = pd.DataFrame(contexts.reset_index(drop=True)).assign(
        act=actions.reset_index(drop=True)).assign(r=rewards.reset_index(drop=True)).assign(wgt=imp_wgt)
    
    rewards_linreg, rewards_linreg_iw = pd.DataFrame(), pd.DataFrame()
    rewards_rf, rewards_rf_iw = pd.DataFrame(), pd.DataFrame()
    
    for act in sorted(list(set(actions))):
        df = merged.loc[merged["act"]==act]
        X, R, wgt = df.drop(columns=["act","r", "wgt"]), df["r"], df["wgt"]
        
        # Direct method with linear ridge regression
        rewards_linreg[act] = RidgeCV([1e-3, 1e-2, 1e-1]).fit(X,R).predict(contexts)
        rewards_linreg_iw[act] = RidgeCV([1e-3, 1e-2, 1e-1]).fit(X,R, sample_weight=wgt).predict(contexts)
        
        # Direct method with a non-linear reward predictor
        rf = RandomForestRegressor()
        params = {'n_estimators': [50, 100], 
                  'max_depth': [5, 10, 20], 
                  'min_samples_split': [2, 5, 10]}

        rewards_rf[act] = GridSearchCV(rf, params, cv=3).fit(X,R).predict(contexts)
        rewards_rf_iw[act] = GridSearchCV(rf, params, cv=3).fit(X,R, sample_weight=wgt).predict(contexts)
        
    act_dist = policy.get_action_distribution(contexts)
    
    est["dr-lin"] = (rewards_linreg * act_dist).sum().sum() / act_dist.shape[0]
    est["dr-iw-lin"] = (rewards_linreg_iw * act_dist).sum().sum() / act_dist.shape[0]
    est["dr-rf"] = (rewards_rf * act_dist).sum().sum() / act_dist.shape[0]
    est["dr-iw-rf"] = (rewards_rf_iw * act_dist).sum().sum() / act_dist.shape[0]
    
    return est

In [13]:
def get_estimator_stats(estimates, true_parameter_value=None):
    est_stat = []
    for est in estimates.columns:
        pred_means = estimates[est]
        stat = {}
        stat['stat'] = est
        stat['mean'] = np.mean(pred_means)
        stat['SD'] = np.std(pred_means)
        stat['SE'] = np.std(pred_means) / np.sqrt(len(pred_means))
        if true_parameter_value:
            stat['bias'] = stat['mean'] - true_parameter_value
            stat['RMSE'] = np.sqrt(np.mean((pred_means - true_parameter_value) ** 2))
        est_stat.append(stat)

    return pd.DataFrame(est_stat)

#### picked stochastic policy here (deterministic policy has risk of getting 0 weights in the propensities)

In [14]:
def value_est_output(data, model, trials=5, rng=default_rng(7), eps=0.5):
    
    n = data.shape[0]
    train_frac = 0.7
    train_size = round(train_frac * n)
    train_idx = rng.choice(n, size = train_size, replace = False)
    test_idx = np.setdiff1d(np.arange(n), train_idx, assume_unique=True)

    data_context, data_label = data.drop("label", axis=1), data["label"]

    X_train, y_train = data_context.iloc[train_idx], data_label.iloc[train_idx]
    X_test, y_test = data_context.iloc[test_idx], data_label.iloc[test_idx]

    full_rewards = create_rewards(data, "label", True)
    full_rewards_test = full_rewards.iloc[test_idx].drop("label", axis=1)

    model.fit(X_train, y_train)
    policy_stochastic = SKLearnPolicy(model=model, num_actions=len(data.label.unique()), is_deterministic=False)
    policy_true_value = policy_stochastic.get_value_estimate(X_test, full_rewards_test)
    
    classes, k = model.classes_, len(model.classes_)
    logging_policy = BanditLoggingPolicy(num_actions=k, eps=eps, actions=y_test, classes=classes)
    logging_policy_value = logging_policy.get_value_estimate(X=X_test, full_rewards=full_rewards_test)   
    print(f"Logging policy value est: {logging_policy_value:.6f}")
    print(f"Target policy true value: {policy_true_value:.6f}")
    
    val_ests = []    
    for i in range(trials):
        contexts, actions, rewards, propensities = generate_bandit_feedback(X_test, full_rewards_test, logging_policy, rng=rng)
        est = get_value_estimators(policy_stochastic, contexts, actions, rewards, propensities)
        val_ests.append(est)
    df = pd.DataFrame(val_ests)
    
    return get_estimator_stats(df, true_parameter_value=policy_true_value) 

### Yeast data results

- \# of classes = 10
- Sample size = 1,484


In [16]:
from itertools import product
lr = LogisticRegression(multi_class='multinomial')
gb = GradientBoostingClassifier()

params = {'model_list': [lr, gb], 
          'eps': [0.1, 0.5, 0.9]}
keys = params.keys()
values = (params[key] for key in keys)
combinations = [dict(zip(keys, combination)) for combination in product(*values)]

for c in combinations:
    print(f"Model: {c['model_list']} | eps: {c['eps']}") 
    display(value_est_output(yeast_data, model=c['model_list'], eps=c['eps']))

Model: LogisticRegression(multi_class='multinomial') | eps: 0.1
Logging policy value est: 0.182022
Target policy true value: 0.314607


,stat,mean,SD,SE,bias,RMSE
0,mean,0.183820,0.018389,0.008224,-0.130787,0.132073
1,iw,0.348268,0.054168,0.024225,0.033661,0.063775
2,sn-iw,0.277578,0.043843,0.019607,-0.037029,0.057388
3,dr-lin,0.449304,0.023323,0.010430,0.134697,0.136702
4,dr-iw-lin,0.338234,0.046945,0.020995,0.023627,0.052556
5,dr-rf,0.441459,0.026136,0.011689,0.126852,0.129517
6,dr-iw-rf,0.436737,0.029451,0.013171,0.122130,0.125631


Model: LogisticRegression(multi_class='multinomial') | eps: 0.5
Logging policy value est: 0.566292
Target policy true value: 0.350562


,stat,mean,SD,SE,bias,RMSE
0,mean,0.554607,0.028261,0.012639,0.204045,0.205993
1,iw,0.351342,0.022787,0.010191,0.000780,0.022801
2,sn-iw,0.285353,0.036989,0.016542,-0.065208,0.074969
3,dr-lin,0.720666,0.018441,0.008247,0.370104,0.370563
4,dr-iw-lin,0.376881,0.036885,0.016495,0.026320,0.045312
5,dr-rf,0.695782,0.020236,0.009050,0.345220,0.345813
6,dr-iw-rf,0.708820,0.022593,0.010104,0.358259,0.358970


Model: LogisticRegression(multi_class='multinomial') | eps: 0.9
Logging policy value est: 0.910112
Target policy true value: 0.323596


...

,stat,mean,SD,SE,bias,RMSE
0,mean,0.202697,0.013363,0.005976,-0.361348,0.361595
1,iw,0.583855,0.051384,0.022980,0.019810,0.055071
2,sn-iw,0.495715,0.050007,0.022364,-0.068330,0.084674
3,dr-lin,0.562073,0.022375,0.010007,-0.001972,0.022462
4,dr-iw-lin,0.543016,0.057749,0.025826,-0.021029,0.061458
5,dr-rf,0.572912,0.031789,0.014217,0.008867,0.033003
6,dr-iw-rf,0.573462,0.027067,0.012105,0.009417,0.028658


Model: GradientBoostingClassifier() | eps: 0.5
Logging policy value est: 0.555056
Target policy true value: 0.514607


,stat,mean,SD,SE,bias,RMSE
0,mean,0.561798,0.020299,0.009078,0.047191,0.051372
1,iw,0.515864,0.022011,0.009843,0.001258,0.022047
2,sn-iw,0.480446,0.059281,0.026511,-0.034161,0.068419
3,dr-lin,0.815110,0.014744,0.006594,0.300504,0.300865
4,dr-iw-lin,0.596545,0.052003,0.023256,0.081938,0.097047
5,dr-rf,0.834520,0.016715,0.007475,0.319914,0.320350
6,dr-iw-rf,0.828061,0.013672,0.006114,0.313454,0.313752


Model: GradientBoostingClassifier() | eps: 0.9
Logging policy value est: 0.925843
Target policy true value: 0.485393


,stat,mean,SD,SE,bias,RMSE
0,mean,0.913258,0.006130,0.002741,0.427865,0.427909
1,iw,0.502134,0.004604,0.002059,0.016741,0.017362
2,sn-iw,0.460138,0.077432,0.034628,-0.025256,0.081446
3,dr-lin,0.958369,0.005605,0.002506,0.472976,0.473009
4,dr-iw-lin,0.749983,0.065861,0.029454,0.264589,0.272663
5,dr-rf,0.959388,0.005181,0.002317,0.473995,0.474023
6,dr-iw-rf,0.957813,0.008317,0.003719,0.472419,0.472493


### Bean data results

- \# of classes = 17
- Sample size = 13,611

In [17]:
for c in combinations:
    print(f"Model: {c['model_list']} | eps: {c['eps']}") 
    display(value_est_output(bean_data, model=c['model_list'], eps=c['eps']))

Model: LogisticRegression(multi_class='multinomial') | eps: 0.1
Logging policy value est: 0.229733
Target policy true value: 0.570659


,stat,mean,SD,SE,bias,RMSE
0,mean,0.229929,0.008344,0.003731,-0.340730,0.340832
1,iw,0.639406,0.023937,0.010705,0.068747,0.072795
2,sn-iw,0.537166,0.021262,0.009509,-0.033493,0.039672
3,dr-lin,0.519783,0.025353,0.011338,-0.050876,0.056843
4,dr-iw-lin,0.578906,0.012443,0.005564,0.008247,0.014927
5,dr-rf,0.605654,0.005903,0.002640,0.034995,0.035490
6,dr-iw-rf,0.601957,0.006981,0.003122,0.031298,0.032067


Model: LogisticRegression(multi_class='multinomial') | eps: 0.5
Logging policy value est: 0.559882
Target policy true value: 0.590987


,stat,mean,SD,SE,bias,RMSE
0,mean,0.574235,0.007257,0.003245,-0.016752,0.018256
1,iw,0.592244,0.004278,0.001913,0.001257,0.004459
2,sn-iw,0.513017,0.017952,0.008028,-0.077970,0.080010
3,dr-lin,0.676561,0.020092,0.008985,0.085574,0.087901
4,dr-iw-lin,0.569759,0.003305,0.001478,-0.021228,0.021484
5,dr-rf,0.676057,0.004703,0.002103,0.085070,0.085200
6,dr-iw-rf,0.654831,0.000931,0.000416,0.063844,0.063850


Model: LogisticRegression(multi_class='multinomial') | eps: 0.9
Logging policy value est: 0.917708
Target policy true value: 0.580456


...

,stat,mean,SD,SE,bias,RMSE
0,mean,0.226892,0.006613,0.002957,-0.674161,0.674194
1,iw,0.961034,0.030751,0.013752,0.059981,0.067404
2,sn-iw,0.879001,0.014513,0.006491,-0.022052,0.026399
3,dr-lin,0.740926,0.012259,0.005482,-0.160127,0.160595
4,dr-iw-lin,0.888522,0.010517,0.004704,-0.012531,0.016360
5,dr-rf,0.885854,0.005676,0.002538,-0.015200,0.016225
6,dr-iw-rf,0.894478,0.005839,0.002611,-0.006575,0.008793


Model: GradientBoostingClassifier() | eps: 0.5
Logging policy value est: 0.564291
Target policy true value: 0.899339


,stat,mean,SD,SE,bias,RMSE
0,mean,0.574137,0.007394,0.003307,-0.325202,0.325286
1,iw,0.908826,0.012696,0.005678,0.009487,0.015849
2,sn-iw,0.868764,0.025474,0.011392,-0.030574,0.039796
3,dr-lin,0.883823,0.023894,0.010686,-0.015516,0.028490
4,dr-iw-lin,0.893475,0.016542,0.007398,-0.005864,0.017551
5,dr-rf,0.955077,0.001068,0.000478,0.055738,0.055748
6,dr-iw-rf,0.952799,0.003058,0.001367,0.053460,0.053547


Model: GradientBoostingClassifier() | eps: 0.9
Logging policy value est: 0.918932
Target policy true value: 0.897134


,stat,mean,SD,SE,bias,RMSE
0,mean,0.911585,0.002992,0.001338,0.014450,0.014757
1,iw,0.892195,0.002461,0.001101,-0.004939,0.005518
2,sn-iw,0.858731,0.056599,0.025312,-0.038404,0.068398
3,dr-lin,0.970896,0.006320,0.002826,0.073761,0.074031
4,dr-iw-lin,0.912289,0.022349,0.009995,0.015154,0.027002
5,dr-rf,0.982769,0.000770,0.000344,0.085635,0.085638
6,dr-iw-rf,0.980414,0.002121,0.000949,0.083279,0.083306


Direct methods performs well under more random logging policy (small epsilon; less bias towards true label) 